# Six-bar mechanism balancing

In [1]:
from differential_evolution import differential_evolution
from BetaShF_population import *
from BetaShM_population import *
import numpy as np
import time

assert (ShM(np.zeros((1, 15)) ) == np.ones((10))).all()
assert (ShF(np.zeros((1, 15)) ) == np.ones((10))).all()
np.random.seed(1)

## Util

In [2]:
def cleanData(samples, fitness, forces, moments):
    filterF = forces < 1
    filterM = moments < 1
    f = np.logical_and(filterF, filterM)
    print(f.shape)
    return samples[f], fitness[f], forces[f], moments[f]

In [3]:
def saveData(samples, fitness, forces, moments):
    now = str(int(time.time()))
    np.savetxt(now + "Population.txt", samples)
    np.savetxt(now + "Fitness.txt", fitness)
    np.savetxt(now + "ShForces.txt", forces)
    np.savetxt(now + "ShMoments.txt", moments)
    return now

In [4]:
def logSample(prefix, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open(name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(prefix + "Population.txt", s)
    appendToFile(prefix + "Fitness.txt", str(fitness))
    appendToFile(prefix + "ShForces.txt", str(force))
    appendToFile(prefix + "ShMoments.txt", str(moment))

## Problem definition

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

$$
f(x) = \alpha ShF(x) + (1 - \alpha) ShM(x)
$$
$$
\therefore \alpha = \frac{f(x) - ShM(x)}{ShF(x) - ShM(x)}
$$

In [5]:
def objective(s, ShF, ShM, alpha):
    return (alpha)*ShF(s) + (1 - alpha)*ShM(s)

## Sample generation

In [6]:
#nSamples, nGen = 1, 500 # Tune here
nSamples, nGen = 1, 1

nVariables = 15
nWeights = nVariables // 3
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
shForces = np.zeros((nSamples))
shMoments = np.zeros((nSamples))

### Differential Evolution

#### Define boundaries

In [7]:
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
print(bounds)

[[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


#### Define pop sizes

In [8]:
popSizes = [50, 150, 225]

In [9]:
startTime = str(int(time.time()))
print("Saving data with the prefix:", startTime)
eTimeDE, goodSols = 0, 0
for pop in popSizes:
    print(f'Results with a popSize = {pop}')
    for s in range(nSamples):
        print(f'Computing sample: {s}')
        alpha = np.random.normal(0.5, 0.20)         # To get values near 0.5
        alpha = np.min([1, np.max([0, alpha])])     # Box constraints [0, 1]
        start = time.perf_counter()
        r = differential_evolution(objective, bounds, popSize = pop, nMax = nGen, args = (ShF, ShM, alpha), cR = 0.7)
        end = time.perf_counter()
        sample, fitness, force, moment = r['x'], r["fun"], ShF(r['x']), ShM(r['x'])
        if force < 1 and moment < 1:
            goodSols += 1
            logSample(f'{startTime}-{pop}-', sample, fitness, force, moment)
        print("Sample ", s, ":\tshF: ", force, "\tshM: ", moment, "\talpha = ", alpha, sep = "")
        eTimeDE += (end - start) #Time in seconds
        print('\r')
    if nSamples: eTimeDE /= nSamples
    print("Average time of execution:", eTimeDE, "seconds. It was run", nSamples, "times.")
    print("Went from ", nSamples, " samples to ", goodSols, " after cleaning data", sep = "")

Saving data with the prefix: 1678168139
Results with a popSize = 50
Computing sample: 0
uno
(50, 3, 15) (50, 15)


/home/lalitor/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()